This notebook is a continuation of the `fim_matching_IEEE14.ipynb`.
Here, we will only look at some area in the network.
Following the paper by Transtrum et al. (https://ieeexplore.ieee.org/abstract/document/8586586), we will consider the area A, consisting of buses 3, 4, 7, 8, and 9.

In [ ]:
from pathlib import Path
import json

import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

from information_matching.convex_optimization import ConvexOpt
from information_matching.utils import tol, eps

%matplotlib inline
plt.style.use("default")

In [ ]:
# Set directories
case = 14  # 14-bus case

WORK_DIR = Path().absolute()
FIM_DIR = WORK_DIR / "FIMs"
DATA_DIR = WORK_DIR / "models" / "data"

# Setup

First, remember that we only want to make measurement of the buses in the area and we only care about the observability of these buses.

In [ ]:
# Load the area data, which contains information about which buses in the area
with open(DATA_DIR / "area_partition.json", "r") as f:
    area_dict = json.load(f)

area = "AreaA"
buses_area = area_dict[area]

In [ ]:
# Model information
nparams = 2 * len(buses_area)

In [ ]:
# Configurations
nconfigs = len(buses_area)
configs = buses_area
config_ids = np.array([f"bus{ii}" for ii in configs])
config_ids

In [ ]:
# Load configuration FIMs
# Since we only care about the buses in the area, we need to remove some columns in
# the Jacobians.
idx_buses = np.array(buses_area) - 1  # buses zero-based index
# Index of the parameters we will use
idx_params = np.sort(np.append(2 * idx_buses, 2 * idx_buses + 1))

fim_configs_tensor = np.empty((nconfigs, nparams, nparams))
for ii, bus in enumerate(configs):
    J = np.load(FIM_DIR / f"jacobian_bus{bus}.npy")[:, idx_params]
    I = J.T @ J
    fim_configs_tensor[ii] = I

In [ ]:
# Target FIM
lambda_tol = 1e-5  # Eigenvalue cutoff
fim_target = np.diag(np.ones(nparams)) * lambda_tol
print("Eigenvalue lower bound:", lambda_tol)

# Convex optimization

In [ ]:
# Convex optimization
# Settings
cvx_tol = eps ** 0.75
solver = dict(verbose=False, solver=cp.SDPA, epsilonStar=cvx_tol)
print("Tolerance:", cvx_tol)

plt.figure()
wopt = np.ones(nconfigs)
colors = ["r", "g", "b", "k"]
for ii in range(10):
    cvxopt = ConvexOpt(
        fim_target,
        fim_configs_tensor,
        np.array(config_ids),
        norm={"weights": wopt + 1e-15},
        l1norm_obj=True,
    )

    try:
        # Solve
        cvxopt.solve(solver=solver)
        result = cvxopt.result.copy()
        wopt = cvxopt.result["wm"]
        dual = cvxopt.result["dual_wm"]
        print("Violation:", cvxopt.constraints[1].violation())
    except Exception:
        cvxopt.result = result
        break

    plt.plot(wopt, c=colors[ii % len(colors)], label=ii)
    plt.plot(dual, c=colors[ii % len(colors)], ls="--")

plt.yscale("log")
plt.xticks(range(nconfigs), config_ids, rotation=90)
plt.ylabel("Weights")
plt.legend(title="Iteration", bbox_to_anchor=(1, 1))
plt.show()

## Post-processing

In [ ]:
# Get the optimal buses
idx_wopt = cvxopt._get_idx_nonzero_wm(np.sqrt(cvx_tol))  # Index to the optimal buse`s
print("Optimal buses:")
_ = [print(f'{config_ids[ii]} \t {wopt[ii]}') for ii in idx_wopt]

In [ ]:
# Plot the optimal configuration FIM
weighted_fims_configs = fim_configs_tensor * wopt.reshape((-1, 1, 1))
I = np.sum(weighted_fims_configs[idx_wopt], axis=0)
print("Eigenvalues:")
print(np.linalg.eigvalsh(I))

plt.figure()
cbound = np.max([abs(np.min(I)), abs(np.max(I))])
plt.imshow(I, vmin=-cbound, vmax=cbound, cmap="bwr")
plt.colorbar()
plt.show()